In [11]:
## IMPORTS
import pandas as pd

TRENDINS_PATH = '../datasets/trendings.json'

In [12]:
def readTrendings():
    try:
        df = pd.read_json(TRENDINS_PATH, encoding='utf-8')
        return df
    except ValueError as ve:
        print(ve)
        return None

In [8]:
df = readTrendings()
df.head(10)

,video_name,video_channel_name,video_url,rank_trend,likes,views,comments,trending_start_date,trending_end_date,video_channel_account,keywords
0,"Anúncio de Plata, Alcaraz no CT e Pedro no cam...",FLA TV,https://www.youtube.com/watch?v=aQlxkX11Z8U,"[#6, #8]",14176,155632,257.0,1725062400000,1725148800000,@flamengo,"[flatv, fla tv, flamengo, Mengo, Mengão, Futeb..."
1,O mistério do jogo mais macabro do Roblox,Goularte,https://www.youtube.com/watch?v=XoCeJhACk8E,"[#5, #5]",38038,375723,1553.0,1725062400000,1725148800000,@G0ularte,"[g0ularte, goularte, comentários]"
2,MC Luuky - Nem Ligo pro Amor 💔 (Clipe Oficial)...,MC Luuky,https://www.youtube.com/watch?v=aRu1y9A4orU,"[#16, #14]",19345,332533,8135.0,1725062400000,1725148800000,@mcluuky,"[MC Luuky, Desacato, Te Levei pro Beco, Vou pa..."
3,Grupo Menos é Mais - Coração Partido (Clipe Of...,Grupo Menos é Mais,https://www.youtube.com/watch?v=n5wxZ_OBUXk,"[#6, #4]",35897,1234773,1241.0,1725062400000,1725148800000,@GrupoMenosEMais,"[pagode, recaída, vai me dando corda, cantor d..."
4,FLOOR 2 ENDING + ALL CUTSCENES & BOSSES! - DOO...,Rizado,https://www.youtube.com/watch?v=IXFKvV0hx20,"[#8, #7]",42133,2056323,3960.0,1725062400000,1725148800000,@RizadoGames,[]
5,skibidi toilet 77 (part 2),DaFuq!?Boom!,https://www.youtube.com/watch?v=x1HMTr0mp8U,"[#2, #1]",628782,9206247,48373.0,1725062400000,1725148800000,@DaFuqBoom,[]
6,"Raphinha brilha com hat-trick, e Barcelona atr...",ESPN Brasil,https://www.youtube.com/watch?v=TwrJup8dY1w,"[#1, #3]",69967,1188512,3354.0,1725062400000,1725148800000,@espnbrasil,"[Futebol, Brasil, FUTEBOL BRASILEIRO, EUROPA L..."
7,Qualifying Highlights | 2024 Italian Grand Prix,FORMULA 1,https://www.youtube.com/watch?v=0ZAc44H69wY,"[#3, #2]",75299,3313281,2771.0,1725062400000,1725148800000,@Formula1,"[F1, Formula One, Formula 1]"
8,Discover the BEST Autumn Festivities in Hay Day!,Hay Day,https://www.youtube.com/watch?v=WaVyT0fcSkw,"[#12, #10]",14795,631021,804.0,1725062400000,1725148800000,@hayday,"[hay day, iphone, ipad, android, game, app, su..."
9,Liniker - CAJU (Visual Experience),Liniker,https://www.youtube.com/watch?v=9nbYJm9FYa0,"[#17, #15]",2198,48461,56.0,1725062400000,1725148800000,@oficialliniker,[]


In [38]:
def filterMatchChannels(keyword):
    df = readTrendings()
    channels = []
    ## IF DATAFRAME IS NOT RECEIVE NONE
    if not df.empty:
        ## LOOP OBSERVATIONS IN DATAFRAME
        for index, row in df.iterrows():
            ## CHECK IF WORD APPEAR IN KEYWORDS CHANNEL
            if keyword.lower() in [x.lower() for x in row['keywords']]:
                ## if not exist in channel, append de value
                if row['video_channel_account'] not in channels:
                    channels.append(row['video_channel_account'])
    
    return channels

In [39]:
channels = filterMatchChannels('Futebol')
channels

['@flamengo',
 '@espnbrasil',
 '@EsportenaBand',
 '@geglobo',
 '@Palmeiras',
 '@jovempanesportes',
 '@LibertadoresBR',
 '@TNTSportsBR',
 '@Banheiristas',
 '@scinternacional',
 '@BotafogoTV',
 '@DeSolaOficial',
 '@CBF',
 '@Sportimperativo',
 '@fluminensefc',
 '@MauroCezar',
 '@corinthians',
 '@CasalTorcedorReact']